In [1]:
import os

celebd_df_v2_path = "/home/roshen/torch/archive" 
print(os.listdir(celebd_df_v2_path))


['YouTube-real', 'deepFake_Detection_using_densenet121 vs code.ipynb', 'List_of_testing_videos.txt', 'restney152v2-lstm.ipynb', 'Celeb-real', 'resnet50.ipynb', 'Celeb-synthesis', 'deepfake_detection_model (1).h5']


In [6]:
import pandas as pd


In [7]:

celebd_df_v2_path = "/home/roshen/torch/archive" 

video_paths = []
labels = []

youtube_real_path = os.path.join(celebd_df_v2_path, "/home/roshen/torch/archive/YouTube-real")
for video_file in os.listdir(youtube_real_path):
    video_paths.append(os.path.join(youtube_real_path, video_file))
    labels.append('REAL')

celeb_synthesis_path = os.path.join(celebd_df_v2_path, "/home/roshen/torch/archive/Celeb-synthesis")
for video_file in os.listdir(celeb_synthesis_path):
    video_paths.append(os.path.join(celeb_synthesis_path, video_file))
    labels.append('FAKE')

celeb_real_path = os.path.join(celebd_df_v2_path, "/home/roshen/torch/archive/Celeb-real")
for video_file in os.listdir(celeb_real_path):
    video_paths.append(os.path.join(celeb_real_path, video_file))
    labels.append('REAL')

df = pd.DataFrame({'video_path': video_paths, 'label': labels})

print(df.head())


                                          video_path label
0  /home/roshen/torch/archive/YouTube-real/00051.mp4  REAL
1  /home/roshen/torch/archive/YouTube-real/00050.mp4  REAL
2  /home/roshen/torch/archive/YouTube-real/00122.mp4  REAL
3  /home/roshen/torch/archive/YouTube-real/00197.mp4  REAL
4  /home/roshen/torch/archive/YouTube-real/00222.mp4  REAL


In [8]:
import cv2

def extract_frames(video_path, num_frames=10):
    frames = []
    cap = cv2.VideoCapture(video_path)
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = total_frames // num_frames
    
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames

sample_video = df.video_path[0] 
frames = extract_frames(sample_video)

print(len(frames))


10


In [9]:
import numpy as np

def preprocess_frames(frames, target_size=(224, 224)):
    processed_frames = []
    for frame in frames:
        frame = cv2.resize(frame, target_size)
        frame = frame.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
        processed_frames.append(frame)
    return np.array(processed_frames)

processed_frames = preprocess_frames(frames)
print(processed_frames.shape)  # Should show (num_frames, height, width, channels)


(10, 224, 224, 3)


In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

def create_resnet_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)  # Binary classification
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_resnet_model()
model.summary()


2024-10-08 14:21:18.994199: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-08 14:21:19.260639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-08 14:21:19.340871: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-08 14:21:19.369694: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-08 14:21:19.595625: I tensorflow/core/platform/cpu_feature_guar

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 84s 1us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

***REAL-1***

In [11]:
import os
import cv2
import numpy as np

def extract_frames_from_videos(video_paths, target_folder, num_frames=10):
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    for video_path in video_paths:
        video_name = os.path.basename(video_path).split('.')[0]
        cap = cv2.VideoCapture(video_path)

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        frame_interval = total_frames // num_frames
        frames = []

        for i in range(num_frames):
            cap.set(cv2.CAP_PROP_POS_FRAMES, i * frame_interval)
            ret, frame = cap.read()
            if ret:
                frame = cv2.resize(frame, (224, 224))
                frames.append(frame)
                frame_filename = os.path.join(target_folder, f"{video_name}_frame_{i}.jpg")
                cv2.imwrite(frame_filename, frame)

        cap.release()

video_paths = df['video_path'].tolist()  # List of video paths from your DataFrame
extract_frames_from_videos(video_paths, 'extracted_frames')


In [12]:
frame_paths = []
labels = []

for video_path in video_paths:
    video_name = os.path.basename(video_path).split('.')[0]
    for i in range(10):
        frame_paths.append(f'extracted_frames/{video_name}_frame_{i}.jpg')
        labels.append(df[df['video_path'] == video_path]['label'].values[0])

frame_df = pd.DataFrame({'frame_path': frame_paths, 'label': labels})


In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    dataframe=frame_df,
    x_col='frame_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=128,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=frame_df,
    x_col='frame_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=6128,
    class_mode='binary',
    subset='validation'
)

Found 52225 validated image filenames belonging to 2 classes.


/home/roshen/torch/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 9 invalid image filename(s) in x_col="frame_path". These filename(s) will be ignored.
  warnings.warn(


Found 13056 validated image filenames belonging to 2 classes.


/home/roshen/torch/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 9 invalid image filename(s) in x_col="frame_path". These filename(s) will be ignored.
  warnings.warn(


In [38]:
import os

for path in frame_df['frame_path']:
    if not os.path.exists(path):
        print(f"Missing frame: {path}")


Missing frame: extracted_frames/id27_0005_frame_1.jpg
Missing frame: extracted_frames/id27_0005_frame_2.jpg
Missing frame: extracted_frames/id27_0005_frame_3.jpg
Missing frame: extracted_frames/id27_0005_frame_4.jpg
Missing frame: extracted_frames/id27_0005_frame_5.jpg
Missing frame: extracted_frames/id27_0005_frame_6.jpg
Missing frame: extracted_frames/id27_0005_frame_7.jpg
Missing frame: extracted_frames/id27_0005_frame_8.jpg
Missing frame: extracted_frames/id27_0005_frame_9.jpg


In [39]:
videos_to_exclude = ['id27_0005']


In [40]:
filtered_frame_df = frame_df[~frame_df['frame_path'].str.contains('|'.join(videos_to_exclude))]


In [41]:
train_generator = datagen.flow_from_dataframe(
    dataframe=filtered_frame_df,
    x_col='frame_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=128,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=filtered_frame_df,
    x_col='frame_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=128,
    class_mode='binary',
    subset='validation'
)


Found 52168 validated image filenames belonging to 2 classes.
Found 13042 validated image filenames belonging to 2 classes.


In [42]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [43]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)


/home/roshen/torch/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
408/408 ━━━━━━━━━━━━━━━━━━━━ 1938s 5s/step - accuracy: 0.8790 - loss: 0.7187 - val_accuracy: 0.7700 - val_loss: 0.5308
Epoch 2/5
408/408 ━━━━━━━━━━━━━━━━━━━━ 0s 287us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/5


2024-10-08 18:13:28.452409: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)
2024-10-08 18:13:28.481797: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


408/408 ━━━━━━━━━━━━━━━━━━━━ 1929s 5s/step - accuracy: 0.8856 - loss: 0.3538 - val_accuracy: 0.7701 - val_loss: 0.4658
Epoch 4/5
408/408 ━━━━━━━━━━━━━━━━━━━━ 0s 92us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5


2024-10-08 18:45:37.859791: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


408/408 ━━━━━━━━━━━━━━━━━━━━ 1933s 5s/step - accuracy: 0.8884 - loss: 0.3376 - val_accuracy: 0.7700 - val_loss: 0.4691


In [44]:
model.save('deepfake_detection_model.h5')


In [51]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

elebd_df_v2_path = "/home/roshen/torch/archive" 

video_paths = []
labels = []

youtube_real_path = os.path.join(celebd_df_v2_path, "/home/roshen/torch/archive/YouTube-real")
for video_file in os.listdir(youtube_real_path):
    video_paths.append(os.path.join(youtube_real_path, video_file))
    labels.append('REAL')

celeb_synthesis_path = os.path.join(celebd_df_v2_path, "/home/roshen/torch/archive/Celeb-synthesis")
for video_file in os.listdir(celeb_synthesis_path):
    video_paths.append(os.path.join(celeb_synthesis_path, video_file))
    labels.append('FAKE')

celeb_real_path = os.path.join(celebd_df_v2_path, "/home/roshen/torch/archive/Celeb-real")
for video_file in os.listdir(celeb_real_path):
    video_paths.append(os.path.join(celeb_real_path, video_file))
    labels.append('REAL')

df = pd.DataFrame({'video_path': video_paths, 'label': labels})

print(df.head())

# Data Augmentation for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # 20% of the data for validation
)

# Set reduced batch size
train_generator = datagen.flow_from_dataframe(
    dataframe=frame_df,
    x_col='frame_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=16,  # Reduced batch size to 16
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=frame_df,
    x_col='frame_path',
    y_col='label',
    target_size=(224, 224),
    batch_size=16,  # Reduced batch size for validation as well
    class_mode='binary',
    subset='validation'
)

# Build the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent training
for layer in base_model.layers:
    layer.trainable = False

# Build the final model by adding custom layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # For binary classification

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with reduced epochs (5) and batch size (16)
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5,  # Reduced epochs to 5
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)

# Save the trained model
model.save('deepfake_detection_model.h5')


                                          video_path label
0  /home/roshen/torch/archive/YouTube-real/00051.mp4  REAL
1  /home/roshen/torch/archive/YouTube-real/00050.mp4  REAL
2  /home/roshen/torch/archive/YouTube-real/00122.mp4  REAL
3  /home/roshen/torch/archive/YouTube-real/00197.mp4  REAL
4  /home/roshen/torch/archive/YouTube-real/00222.mp4  REAL
Found 52225 validated image filenames belonging to 2 classes.


/home/roshen/torch/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 9 invalid image filename(s) in x_col="frame_path". These filename(s) will be ignored.
  warnings.warn(


Found 13056 validated image filenames belonging to 2 classes.


/home/roshen/torch/lib/python3.10/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 9 invalid image filename(s) in x_col="frame_path". These filename(s) will be ignored.
  warnings.warn(


Epoch 1/5


...

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


3265/3265 ━━━━━━━━━━━━━━━━━━━━ 2672s 818ms/step - accuracy: 0.8884 - loss: 0.3850 - val_accuracy: 0.7702 - val_loss: 0.5686
Epoch 4/5
3265/3265 ━━━━━━━━━━━━━━━━━━━━ 0s 88us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/5


2024-10-08 21:34:28.687401: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


3265/3265 ━━━━━━━━━━━━━━━━━━━━ 2595s 795ms/step - accuracy: 0.8874 - loss: 0.3808 - val_accuracy: 0.7702 - val_loss: 0.5612


: 